In [2]:
""" 
1. 环境
---
Google Colab GPU 进行下面方式安装，自己电脑可以按照自己方式进行安装；
YOLOX 安装 0.3.0 分支版本，不要使用仓库最新代码，可能存在未知无法解决的错误；
"""

%cd /content/
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git
%cd YOLOX
# 切换到 0.3.0 版本
!git checkout -b 0.3.0 0.3.0
%pip install -U pip && pip install -r requirements.txt
%pip install -v -e . 


/content
Cloning into 'YOLOX'...
remote: Enumerating objects: 1850, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 1850 (delta 17), reused 33 (delta 7), pack-reused 1782
Receiving objects: 100% (1850/1850), 6.90 MiB | 6.24 MiB/s, done.
Resolving deltas: 100% (1082/1082), done.
/content/YOLOX
Switched to a new branch '0.3.0'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
"""
2. 模型
---
使用 yolox_s 模型，进行迁移学习，可使用 wget 进行下载
"""

#---------------------------------------------
# Google Drive 挂载内容，本地运行时，可注释
#---------------------------------------------
%cd /content/drive/MyDrive/my-object-detection/yolox-crack-demo
!pip install wget

def download_models():
    """  
    下载 yolox_s 模型
    """
    import os
    import wget
    if not os.path.exists('models/yolox_s.pth'):
        os.mkdir('models')
        wget.download('https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_s.pth','models')
    print('models/yolox_s.pth')


download_models()

/content/drive/MyDrive/my-object-detection/yolox-crack-demo
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=e3447431df4489819b051b92583dc2a8732f43a2f72018cb21c2e6c0bdbf748b
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
models/yolox_s.pth


In [6]:
"""
3. 数据集
---
使用 labelimg 进行图片标注，将标注文件和图片放到一个文件夹中
下载 Google Drive Datasets 放到该文件夹，并解压
数据集: https://drive.google.com/file/d/12n1GVrUX4yJHUCDOx-4CPoA2ooiSU5L2/view?usp=share_link
"""

#---------------------------------------------
# Google Drive 挂载内容，本地运行时，可注释
#---------------------------------------------
%cd /content/drive/MyDrive/my-object-detection/yolox-crack-demo

def datasets_download():
    """
    下载数据集 
    """
    import os 
    if not  os.path.exists('datasets'):
        os.mkdir('datasets')
    # 若使用 colab ，可直接挂载 drive ，进行复制操作；否则自行下载，并进行解压
    if not os.path.exists('datasets/crack-datasets'):
        os.system('cp /content/drive/MyDrive/my-datasets/crack-datasets.zip ./datasets/')
        # 解压，可自行进行下载，解压到 datasets 文件夹
        os.system('unzip -d datasets/crack-datasets datasets/crack-datasets.zip')
        # 删除，多余文件
        os.system('rm datasets/crack-datasets.zip')

    
# 下载
datasets_download()

"""
数据整理
"""
#%cd /content/drive/MyDrive/my-object-detection/yolox-crack-demo
#%mv ./datasets/crack-datasets/photo/*.jpg ./datasets/crack-datasets/
#%mv ./datasets/crack-datasets/photo/*.xml ./datasets/crack-datasets/
#%mv -r ./datasets/crack-datasets/photo
#%rm ./datasets/crack-datasets.zip

SyntaxError: ignored

In [26]:
"""
4. 数据集转换 
---
将 labelimg 标注数据集，转换为 coco 数据集，使用 tools/labelimg2coco.py 
"""

%cd /content/drive/MyDrive/my-object-detection/yolox-crack-demo
%rm -r datasets/crack

def datasets_init_coco():
    import os 
    if not  os.path.exists('datasets/crack'):
        os.mkdir('datasets/crack')
    from tools import labelimg2coco
    labelimg2coco(
        labelimg_dir='datasets/crack-datasets',
        coco_dir='datasets/crack',
        classes=['0'],
        image_suffix='.jpg'
    )

datasets_init_coco()

/content/drive/MyDrive/my-object-detection/yolox-crack-demo
datasets/crack/annotations/instances_train2017.json
datasets/crack/annotations/instances_val2017.json
datasets/crack/annotations/instances_test2017.json
datasets/crack/train.txt
datasets/crack/val.txt
datasets/crack/test.txt
datasets/crack/class.txt
train size 28 , val size 8 , test size 4


In [27]:
"""
5. 训练 
---
复制 YOLOX/exps/example/custom/yolox_s.py 文件，并进行变更内容，见 crack_exp_yolox_s.py 
Google Colab 记得清理下 YOLOX_outputs 下的相关文件
"""


%cd /content/drive/MyDrive/yolox-crack-demo/

# %rm -r YOLOX_outputs/crack_exp_yolox_s

!python train.py -f crack_exp_yolox_s.py -d 1 -b 16 --fp16 -o -c models/yolox_s.pth



[Errno 2] No such file or directory: '/content/drive/MyDrive/yolox-crack-demo/'
/content/drive/MyDrive/my-object-detection/yolox-crack-demo
2023-02-21 04:14:41 | INFO     | yolox.core.trainer:130 - args: Namespace(batch_size=16, cache=None, ckpt='models/yolox_s.pth', devices=1, dist_backend='nccl', dist_url=None, exp_file='crack_exp_yolox_s.py', experiment_name='crack_exp_yolox_s', fp16=True, logger='tensorboard', machine_rank=0, name=None, num_machines=1, occupy=True, opts=[], resume=False, start_epoch=None)
2023-02-21 04:14:41 | INFO     | yolox.core.trainer:131 - exp value:
╒═══════════════════╤════════════════════════════╕
│ keys              │ values                     │
╞═══════════════════╪════════════════════════════╡
│ seed              │ None                       │
├───────────────────┼────────────────────────────┤
│ output_dir        │ './YOLOX_outputs'          │
├───────────────────┼────────────────────────────┤
│ print_interval    │ 10                         │
├───────

In [28]:
"""
6. 测试 
---
测试图片在 asserts/demo.jpg , 使用训练完毕的模型进行测试
训练完毕的模型地址：/YOLOX_outputs/best_ckpt.pth
需要修改 demo.py 中的 COCO_CLASS 内容
"""

# 修改 demo.py/COCO_CLASS 内容
# 复制最新模型到 models 文件夹
%cd /content/drive/MyDrive/my-object-detection/yolox-crack-demo
%cp ./YOLOX_outputs/crack_exp_yolox_s/best_ckpt.pth ./models/


/content/drive/MyDrive/my-object-detection/yolox-crack-demo


In [31]:
# 图片推理测试
%cd /content/drive/MyDrive/my-object-detection/yolox-crack-demo
!python demo.py image -f crack_exp_yolox_s.py -c models/best_ckpt.pth --path asserts/test.jpg --conf 0.25 --nms 0.45 --tsize 640 --save_result --device gpu

/content/drive/MyDrive/my-object-detection/yolox-crack-demo
2023-02-21 04:30:26.839 | INFO     | __main__:main:261 - Args: Namespace(camid=0, ckpt='models/best_ckpt.pth', conf=0.25, demo='image', device='gpu', exp_file='crack_exp_yolox_s.py', experiment_name='crack_exp_yolox_s', fp16=False, fuse=False, legacy=False, name=None, nms=0.45, path='asserts/test.jpg', save_result=True, trt=False, tsize=640)
2023-02-21 04:30:27.194 | INFO     | __main__:main:271 - Model Summary: Params: 8.94M, Gflops: 26.76
2023-02-21 04:30:30.015 | INFO     | __main__:main:284 - loading checkpoint
2023-02-21 04:30:30.306 | INFO     | __main__:main:288 - loaded checkpoint done.
2023-02-21 04:30:32.054 | INFO     | __main__:inference:167 - Infer time: 1.7329s
2023-02-21 04:30:32.063 | INFO     | __main__:image_demo:204 - Saving detection result in ./YOLOX_outputs/crack_exp_yolox_s/vis_res/2023_02_21_04_30_30/test.jpg


In [33]:
%cp ./YOLOX_outputs/crack_exp_yolox_s/vis_res/2023_02_21_04_30_30/test.jpg ./asserts/test_result.jpg

In [34]:
""" 
7. 评估模型
"""
%cd /content/drive/MyDrive/my-object-detection/yolox-crack-demo
!python eval.py -f crack_exp_yolox_s.py -c models/best_ckpt.pth -b 16 -d 1 --conf 0.001 --fp16 --fuse

/content/drive/MyDrive/my-object-detection/yolox-crack-demo
2023-02-21 04:31:38 | INFO     | __main__:139 - Args: Namespace(batch_size=16, ckpt='models/best_ckpt.pth', conf=0.001, devices=1, dist_backend='nccl', dist_url=None, exp_file='crack_exp_yolox_s.py', experiment_name='crack_exp_yolox_s', fp16=True, fuse=True, legacy=False, machine_rank=0, name=None, nms=None, num_machines=1, opts=[], seed=None, speed=False, test=False, trt=False, tsize=None)
2023-02-21 04:31:38 | INFO     | __main__:149 - Model Summary: Params: 8.94M, Gflops: 26.76
2023-02-21 04:31:38 | INFO     | __main__:150 - Model Structure:
YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        